# Descripción del Script (Versión 3)


# Flujo de Ejecución del Script

1.  **Autenticación:**
    *   Inicio de sesión en la **API REST**.
    *   Obtención del cliente `rest_client` con token temporal para operar.

2.  **Configuración y Estructura de Datos:**
    *   **Rediseño de CSV:** Se procesan los archivos de entrada estructurados para soportar explícitamente la jerarquía `Farmacia -> Heladera -> Dispositivo`.
    *   **Gestión de Grupos:** Creación de grupos de entidades para "Farmacias" y nuevos grupos específicos para "Heladeras", facilitando la segmentación.

3.  **Despliegue de Infraestructura (Farmacias y Heladeras):**
    *   **Alta de Customer:** Creación del cliente utilizando el **número de PAMI** como identificador único.
    *   **Creación de Activos:** Basado en el CSV, se generan los activos "Farmacia" y "Heladera" (este último con ID técnico único, ver *Nota 5*).
    *   **Relaciones Jerárquicas:** Se establece la relación de tipo **"Contains"** donde *n* heladeras pertenecen a una Farmacia.

4.  **Gestión y Lógica de Dispositivos:**
    *   **Alta de Dispositivos:** Se crean los dispositivos aplicando la nomenclatura de calibración (Serie + Fecha).
    *   **Vinculación:** Se establece la relación de tipo **"Contains"** donde *n* dispositivos pertenecen a una heladera.
    *   **Asignación de Propiedad:** Se asignan masivamente la Farmacia, sus Heladeras y sus Dispositivos al *customer* correspondiente para garantizar la visibilidad completa.
    *   **Configuración de Alarmas:** Implementación de **Rule Chains** para la gestión de alertas, incluyendo el pre-procesamiento de datos para filtrar falsos positivos por apertura y cierre temporal de heladeras.

5.  **Mantenimiento y Recambio (Ciclo de Vida):**
    *   **Invocación de Obsolescencia:** Se invalida el dispositivo anterior asignándole un token aleatorio.
    *   **Reasignación:** Se procesa la información del cliente (ID Técnico de Heladera + Nombre Nuevo Dispositivo) para generar un CSV de actualización. El script ejecuta la reasignación asegurando que el nuevo sensor apunte a la heladera correcta sin perder el historial del contenedor.

6.  **Usuarios e Integración:**
    *   **Gestión de Usuarios:** Validación y creación de usuarios, asignándolos al *customer*.
    *   **Integración OAuth:** Vinculación de la autenticación con el servidor OAuth de COFA, permitiendo que las credenciales externas se mapeen automáticamente a los recursos del cliente en AKIoT.

---

### Notas Técnicas y Estrategia de Datos

**Nota 1: Identificación y Aislamiento**
Se utiliza el número de **PAMI** como ID único para la creación de *customers*, garantizando aislamiento de datos y alarmas. En el dashboard, el título del *customer* es su ID y la etiqueta de la farmacia es su **CUIT** para saber a quien pertenece, esto último solo por consistencia.

**Nota 2: Jerarquía "Heladera" y Trazabilidad**
El activo "Heladera" centraliza el historial. Al sustituir un sensor, el nuevo se vincula a la misma "Heladera", manteniendo la continuidad de los datos de temperatura independientemente del hardware.

**Nota 3: Visibilidad**
Cada *customer* ve solo sus activos. Las alarmas se envían al correo del responsable con copia a los usuarios. Ese correo debería estar asociado a la cuenta COFA.(Telegram no fue requerido).

**Nota 4: Estándares de Nomenclatura y Etiquetas**
*   **Dispositivos:** 
    - El nombre se compone de `SERIE` + `FECHA_CALIBRACION` (Ej: `AKIOT08FA_19_11_2024`). Esto facilita identificar la vigencia del equipo.
    - El Token que vincula la aplicación con el dispositivo será el IMEI de CHIP NBIOT.
*   **Heladeras (ID Técnico):** Se generan automáticamente con el formato `id_farmacia_heladera_[numero_random]`. Este ID es inmutable y necesario para las gestiones de soporte.
*   **Heladeras (Etiqueta/Label):** El usuario final puede editar la etiqueta del activo (Ej: "Heladera Insulina", "Mostrador") para reconocerla operativamente.

**Nota 5: Protocolo de Recambio**
Para procesar un reemplazo, el cliente debe informar obligatoriamente:
1.  El **ID Técnico de la Heladera** (`id_farmacia_heladera_random`) donde se instalará el equipo.
2.  El **Nombre del Nuevo Dispositivo que ha recibido** (`Serie_Fecha`).
Esta información permite generar el CSV exacto para que el script realice la reasignación sin errores de ubicación.

---

**Tareas Pendientes (RoadMap)**
*   Re diseño de los CSV para soportar la relación `Farmacia -> Heladera -> Dispositivo`.
*   Creación de grupos para heladeras.
*   Crear el activo Farmacia y Heladera en función de los CSV.
*   Relacionar tipo Contain, n heladeras pertenecen a una Farmacia.
*   Relacionar tipo Contain, n dispositivos pertenecen a una heladera.
*   Asignar a ese customer su farmacia, sus heladeras y sus dispositivos.
*   Crear alarmas mediante rules Chains y pre procesar para que no se dispare alarmas por apertura y cierre de heladera.
*   Integración de autenticación con servidor OAuth de COFA. Asociar usuarios a sus customer correspondientes

---


# Requerimientos de Integración de Datos (COFA - AKIoT)

Para lograr la integración de la metadata de COFA con la plataforma AKIoT, se requiere la siguiente estructura de información.

> **Nota:** Los campos marcados en *cursiva* son generados o asignados automáticamente por el software por defecto.

## 1. Activo Principal: Farmacia
Datos necesarios para dar de alta la entidad farmacia y su responsable.

*   **Información General:**
    *   Nombre de la Farmacia.
    *   Identificador único de Farmacia.
    *   CUIT de la Farmacia.
*   **Ubicación:**
    *   Dirección (Calle y número).
    *   Ciudad.
    *   Provincia.
    *   Código Postal.
    *   Geolocalización (Latitud y Longitud).
*   **Datos del Responsable (Customer):**
    *   Nombre y Apellido del responsable.
    *   Cargo dentro de la Farmacia.
    *   Correo electrónico (Vinculado al login en la plataforma de COFA).
    *   Número de celular.

## 2. Activo Secundario: Heladera
Nivel jerárquico intermedio para agrupación de sensores.

*   **Definición:** Cantidad de heladeras a monitorear por cada identificador único de farmacia, osea cuantas heladeras hay dentro de una cada farmacia.
*   **Identificadores:**
    *   *Nombre del Activo:* ID técnico generado aleatoriamente por el sistema.
    *   Etiqueta (Label): Valor por defecto. Este campo es editable por el farmacéutico o responsable para facilitar la identificación operativa del punto de monitoreo.

## 3. Dispositivos
Sensores físicos de telemetría.

*   **Definición:** Cantidad de dispositivos asignados (se puede asumir 1 dispositivo por heladera).
*   **Identificadores y Atributos:**
    *   *Nombre del Dispositivo:* Generado automáticamente por el sistema (según lógica de serie/calibración).
    *   *Etiqueta (Label):* Definida por defecto según la magnitud (ej. Temperatura).
    *   *Punto de Monitoreo (Atributo):* Hereda o replica la etiqueta de la heladera para contexto visual (aunque la jerarquía ya vincula el dispositivo a su contenedor).

## 4. Usuarios
- COFA con su servidor de OAuth debería enviar en el JWT (header.payload.signature)
    - Correo de usuario
    - Identificador único de farmacia
    - Clave publica para verificar del emisor
- De forma tal podamos asociar ese usuario a ese customer para que pueda ver sus activos y no los de otros.

## 1. Realizamos el login en la API REST

In [7]:
import logging
from json import load
# Importing models and REST client class from Professional Edition version
from tb_rest_client.rest_client_pe import *
from tb_rest_client.rest import ApiException


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# ThingsBoard REST API URL
url = "https://development.akiot.es/"

# Default Tenant Administrator credentials
username = "tenant@akiot.es"
password = "tenant"


## 2. Nos autenticamos y obtenemos el actual usuario

In [8]:
with RestClientPE(base_url=url) as rest_client:
    try:
        # Auth with credentials
        rest_client.login(username=username, password=password)

        # Getting current user
        current_user = rest_client.get_user()
        logging.info('Current user:\n%r\n', current_user)

    except ApiException as e:
            logging.exception(e)

2025-11-19 19:34:26 - INFO - 470615400 - 8 - Current user:
{'additional_info': {'homeDashboardHideToolbar': True,
                     'lastLoginTs': None,
                     'userActivated': True,
                     'userCredentialsEnabled': True},
 'authority': 'TENANT_ADMIN',
 'created_time': 1760987466533,
 'custom_menu_id': None,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '13814000-1dd2-11b2-8080-808080808080'},
 'email': 'tenant@akiot.es',
 'first_name': None,
 'id': {'entity_type': 'USER', 'id': '87789550-ade8-11f0-b1bc-33c092a70dea'},
 'last_name': None,
 'name': 'tenant@akiot.es',
 'owner_id': {'entity_type': 'TENANT',
              'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'phone': None,
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'version': 2}



## 3. Creamos Grupos para distintas entidades

In [ ]:
# Creo un GRUPO de activos en el nivel de tenant: "farmacias_cofa"
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "ASSET"
group_name = "farmacias_cofa"

try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_asset_group = EntityGroup(name=group_name, type=group_type)
            shared_asset_group = rest_client.save_entity_group(shared_asset_group)
            logging.info('Asset group created:\n%r\n', shared_asset_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba









In [ ]:
# Creamos Grupo de Customers "COFA" a nivel de Tenant
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "CUSTOMER"
group_name = "COFA"

try:
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    shared_customer_group = existing_group
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_customer_group = EntityGroup(name=group_name, type=group_type)
            shared_customer_group = rest_client.save_entity_group(shared_customer_group)
            logging.info('Asset group created:\n%r\n', shared_customer_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba

 ## 4. Creamos un perfil de activos

In [ ]:
# Creo un perfil para los activos "Farmacias COFA"
from tb_rest_client.models.models_pe import EntityId
import logging
from pprint import pprint
profile_name = "Farmacias COFA"
profile_description = "Default asset profile"

# asset_profile = AssetProfile(
#         name=profile_name ,
#         description=profile_description,
#         # Otros campos opcionales:
#         # default_queue_name="Main" 
#         # default=False # No marcar como perfil por defecto
#     )

# saved_asset_profile = rest_client.save_asset_profile(asset_profile)
try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    page_data = rest_client.get_asset_profiles(page_size=100, page=0, text_search=profile_name)

    existing_profile = next((profile for profile in page_data.data if profile.name == profile_name), None)

    if existing_profile:
        logging.info(f"El perfil de activo '{profile_name}' ya existe!!.")
        profile_asset_to_use = existing_profile.id
        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
    else:
    # 1.2 Crear una instancia del modelo AssetProfile
        new_asset_profile = AssetProfile(
                                        name=profile_name,
                                        description=profile_description,)

    # 1.3 Guardar el perfil de activo utilizando el método save_asset_profile
        created_profile = rest_client.save_asset_profile(body=new_asset_profile)

        profile_asset_to_use = created_profile.id
        logging.info(f"✅ Perfil de activo '{profile_name}' creado exitosamente.")
            
except Exception as e:
        logging.error(f"❌ Error al crear/obtener el perfil de activo: {e}")





## 5. Vamos a crear el Customer con sus atributos y crear el activo FARMACIA con sus atributos y se lo vamos a asignar
Por cada activo  (Farmacia) debería Asignar:
- Un Name del activo: Farmacias Garcia 
- Un Label: CUIT o identificador único
- Un Type, perfil de activo: Farmacias COFA
- Una Descripción Opcional
- En esta version se asocia a cada farmacia con su identificador único. No se podrá ver todas las farmacias asociadas al mismo CUIT, ya que este mismo es solo un dato.

### Ejemplo de fila de datos en el CSV
| Name              | Type            | Label | ID_farmacia | calleNumero          | ciudad                         | provincia     | codigoPostal | latitud    | longitud    | CUIT_farmacia   | responsable_farmacias | cargo   | correo             | celular        | chat_id | token_telegram                                           |
|-------------------|-----------------|-------|-------------|-----------------------|--------------------------------|---------------|--------------|------------|-------------|------------------|------------------------|---------|---------------------|----------------|---------|-----------------------------------------------------------|
| Farmacia DEMO 4   | Farmacias COFA  |       | 0004        | Calle 49 884          | La Plata                       | Buenos Aires  | B1900APL     | -34.919722 | -57.954722  | 20-12367315-5    | Carlos Aranda          | Dueño   | caranda@gmail.com   | 549 1177000200 | -12345  | 8491771441:AAFneI93qwYSB_5Rz-WQEB0z6HIlKheWp_c          |
| Farmacia DEMO 5   | Farmacias COFA  |       | 0005        | Avenida Luro 2800     | Mar del Plata                  | Buenos Aires  | B7600GYQ     | -37.994722 | -57.545833  | 30-71223598-8    | Silvia Lopez           | Gerente | slopez@gmail.com    | 549 1125846546 | 1234    | 8491771879:AAFneI93qwYSB_5Rz-WQEB0z6HIlKheWp_c          |
| Farmacia DEMO 6   | Farmacias COFA  |       | 0006        | Avenida Corrientes 5000| Ciudad Autónoma de Buenos Aires| CABA          | C1414AAO     | -34.595833 | -58.435000  | 20-12367315-5    | Soledad Osores         | Gerente | sosores@gmail.com   | 549 1177000200 | -12395  | 6561771441:AAFneI93qwYSB_5Rz-WQEB0z6HIlKheWp_c          |



## ¡IMPORTANTE!, Si borro el Customer se borra toda sus entidades, por ejemplo se borrarían todos sus users, y se perdería la relación con sus activos y dispositivos
Por esta razón se lo debe crear con el siguiente script, con todos sus atributos, users, dispositivos, activos, etc. En un futuro si se quiere modificar sus entidades se hará a través de la appWeb o se crear otro script que lea el customer y modifique lo necesario.

Voy a crear los customers con los identificadores unicos de farmacia, debe tener un chat_id, un token de telegram y un correo asociado a ese identificador. Las alarmas llegaran a ese correo con copia a los correos de los usuarios.

In [ ]:
# Creamos, el customer y lo asociamos a su grupo "COFA", luego creamos el activo para cada fila del CSV, luego establecemos la relación entre ambos.
import csv
from utils import creating_asset
file_path = 'activos_v2.csv'

try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("Name", "")
                tipo = datos.get("Type", "")
                label = datos.get("Label", "")
                id_farmacia = datos.get("ID_farmacia", "")
                calleNumero = datos.get("calleNumero", "")
                ciudad = datos.get("ciudad", "")
                provincia = datos.get("provincia", "")
                codigoPostal = datos.get("codigoPostal", "")
                latitud = datos.get("latitud", "")
                longitud = datos.get("longitud", "")
                cuit_farmacia = datos.get("CUIT_farmacia", "")
                responsable = datos.get("responsable_farmacia", "")
                cargo = datos.get("cargo", "")
                correo = datos.get("correo", "")
                celular = datos.get("celular", "")
                chat_id = datos.get("chat_id", "")
                token_telegram = datos.get("token_telegram", "")

                # Intentamos crear el customers
                try:
                    # 1.1 Intentar obtener todos los customers para verificar si ya existe
                    # page_data = rest_client.get_customers(page_size=100, page=0, text_search="CP1097")
                    page_data = rest_client.get_customers(page_size=5000, page=0)

                    existing_profile_customer = next((profile for profile in page_data.data if profile.name  == id_farmacia), None)
                    # existing_profile_customer = next((profile for profile in page_data.data if profile.address2  == cuit_farmacia), None)
                    # existing_profile_customer = next((profile for profile in page_data.data if profile.email == correo), None)

                    if existing_profile_customer:
                        # SERIA MEJOR FILTRAR POR DNI DEL RESPONSABLE, SOLICITAR QUE LO AGREGUEN AL CSV
                        logging.info(f"El customer con identificador unico de farmacia {id_farmacia} ya existe !!")
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer ya existente
                                                                 
                        # Creating an Asset
                        # Creamos el activo tal que se vea el id_farmacia asociado al customer de farmacia.
                        # Como el identificador de farmacia es único, entonces cada customer tiene un sola farmacia (activo) asociado.
                        # Las alarmas se enviaran solo a ese customer con su correo asociado. 
                        # En el label pongo el CUIT de la farmacia, de esta forma forma podría filtrar que farmacias pertenecen a ese CUIT. aunque el customer es por id_farmacia.
                        creating_asset( rest_client,
                                        name,
                                        cuit_farmacia,
                                        profile_asset_to_use,
                                        existing_profile_customer,
                                        calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                                        latitud,longitud,id_farmacia, responsable, correo, celular)


                         
                        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
                    else:
                        # 1.2 Creamos el customer
                        # Se pone de titulo el id_farmcia unico de la farmacia para identificarlo mas rapido
                        customer1 = Customer(title=id_farmacia,
                                            email=correo,
                                            country="Argentina", # Valor fijo
                                            state=provincia,
                                            city=ciudad,
                                            address=calleNumero,
                                            # address2=cuit_farmacia,  # Uso address2 para guardar el CUIT de la Farmacia y buscar luego
                                            zip=codigoPostal,
                                            phone=celular,)
                                            
                        customer1 = rest_client.save_customer(body=customer1)
                        # Asocio el customer al grupo "COFA"
                        rest_client.add_entities_to_entity_group(shared_customer_group.id, [customer1.id.id])
                        logging.info(f'✅ Customer creado exitosamente: {customer1.title}')
                        # Creo los atributos en el customer
                        # # Veo los atributos del customer
                        # atributos_customer = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                        #                                                                                         entity_type="CUSTOMER",
                        #                                                                                         entity_id=existing_profile_customer.id,
                        #                                                                                         scope="SERVER_SCOPE",
                        #                                                                                         )

                        # 1.3 Cargo nuevos atributos al customer
                        body_to_attributes = {
                            "responsable_farmacia":responsable,
                            "id_farmacia": id_farmacia,
                            "cuit_farmacia": cuit_farmacia,
                            "chat_id": chat_id,
                            "token_telegram": token_telegram,
                            "email": correo # Se usa para las alarmas.

                        }
                        # Guardo un nuevo atributo de servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="CUSTOMER",
                                entity_id=customer1.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        
                        # 1.4 Comparto grupo de dashboard con el customer nuevo creado. Se supone que si ya lo cree ya tiene compartido el Dashboard.
                        page_data = rest_client.get_roles(page_size=1000, page=0,text_search="Entity Group Read-only User")
                        name_rol = page_data.data[0]
                        customer1_users = rest_client.get_entity_group_by_owner_and_name_and_type(customer1.id, "USER", "Customer Users")

                        # Buscamos el grupo de dashboards compartidos Farmacias COFA (a cada Customer luego de crearlo le comparto el Dashboard)
                        current_user = rest_client.get_user()
                        owner_id = current_user.tenant_id
                        group_type = "DASHBOARD"
                        group_name = "Farmacias COFA"

                        try:
                            logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
                            existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
                                    owner_id=owner_id, 
                                    group_type=group_type, 
                                    group_name=group_name
                                )
                            shared_dashboards_group = existing_group
                            logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
                        except ApiException as e:
                                    logging.error(f"❌ Error al buscar el grupo de dashboards: {e}")
                                    logging.exception(e)


                        # Assigning Shared Dashboards to the Customer 1 Users
                        tenant_id = current_user.tenant_id
                        group_permission = GroupPermission(role_id=name_rol.id,
                                                        name="",
                                                        user_group_id=customer1_users.id,
                                                        tenant_id=tenant_id,
                                                        entity_group_id=shared_dashboards_group.id,
                                                        entity_group_type=shared_dashboards_group.type)
                        group_permission = rest_client.save_group_permission(group_permission)
                        logging.info('Group permission created:\n%r\n', group_permission)
                        
                        # 1.6 Creating an Asset

                        creating_asset( rest_client,
                                        name,
                                        cuit_farmacia,
                                        profile_asset_to_use,
                                        customer1,
                                        calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                                        latitud,longitud,id_farmacia, responsable, correo, celular)

                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el customer: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")


## 6. Dispositivos
En esta versión los dispositivos se crean y se relacionan con su farmacia. Como la Farmacia tiene un Customer, se asigna el dispositivo a ese customer. 

In [ ]:
# Vamos a crear los dispositivos asociados a cada activo (Farmacia) y customer (CUIT)
# Si ya existe el dispositivo vamos a listar a quien ya pertenece.
import csv
file_path = 'dispositivos_v2.csv'
name_device_profile = "HC5 NBIoT"
try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("name", "")
                tipo = datos.get("type", "")
                label = datos.get("label", "")
                token_acceso = datos.get("token_acceso", "")
                punto_monitoreo = datos.get("punto_monitoreo", "")
                relacion_farmacia = datos.get("relacion", "")

                # La dirección y el customer se obtienen de la farmacia(Activo)
                # address = datos.get("address", "")
                # customer = datos.get("customer", "")
                try:
                    # 1.1 Intentar obtener el dispositivo por su nombre, ya que debe ser único en AKIoT
                    
                    page_data = rest_client.get_user_devices(page_size=5000, page=0, text_search=name)

                    if page_data.data==[]:
                        logging.info(f"El dispositivo con nombre '{name}' no existe, se puede crear.")
                        # busco el perfil de dispositivo ya creado en AKIoT

                        device_profile = rest_client.device_profile_controller.get_device_profile_infos_using_get(page_size=100, page=0, text_search=name_device_profile)
                        if not device_profile.data:
                            raise Exception(f"No se encontró el perfil de dispositivo con nombre '{name_device_profile}'")
                        
                        
                        
                        asset_to_relate = rest_client.asset_controller.get_all_asset_infos_using_get(page_size=100, page=0,include_customers=True, text_search=relacion_farmacia)
                        # Tengo el customer al que asignare este dispositivo
                        customer_relate_id = asset_to_relate.data[0].customer_id
                        # Tengo la entidad a la que relacionare este dispositivo
                        new_device = Device(name=name, label=label, device_profile_id=device_profile.data[0].id,customer_id=customer_relate_id)
                        new_device = rest_client.save_device(new_device,access_token=token_acceso)
                        logging.info(" Device was created:\n%r\n", new_device)



                        asset_to_relate_id = asset_to_relate.data[0]
                        attributes_from_asset = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                                                                                            entity_type="ASSET",
                                                                                            entity_id=asset_to_relate_id.id,
                                                                                            scope="SERVER_SCOPE",
                                                                                            )
                        attributes_from_asset = {item['key']: item['value'] for item in attributes_from_asset}
                        # Ya tengo los atributos del dispositivo, address, punto_monitoreo, xPos,yPos
                        body_to_attributes = {
                            "address": attributes_from_asset['direccion'],
                            "punto_monitoreo": punto_monitoreo,
                            # "xPos": attributes_from_asset['latitud'],
                            # "yPos": attributes_from_asset['longitud'],
                        }
                        # Guardo los atributos del servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="DEVICE",
                                entity_id=new_device.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        # Creamos la relación entre el activo y el dispositivo
                        relation = EntityRelation(_from=asset_to_relate_id.id, to=new_device.id, type="Contains")
                        rest_client.save_relation(relation)
                        # Asignamos el dispositivo al customer (id_farmacia) correspondiente 

                        # En los activos Farmacia que el title sea el id_farmacia, asi en la lista de dispositivo me queda listado ee nombre de la id_farmacia a la que pertenece, es practico para el buscador
                    else:
                        # Si ya existe el dispositivo vamos a listar a quien ya pertenece.
                        customer_id= page_data.data[0].customer_id
                        try: 
                            customer_name = rest_client.get_customer_by_id(customer_id)
                            logging.info(f"Este dispositivo ya existe y pertenece a la farmacia con CUIT {customer_name.name}")
        
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene customer, pertenece al tenant actual {e}")
                        
                        try:
                            # Buscamos su relacion con algun activo
                            relacion_from = rest_client.find_by_to_v1(page_data.data[0].id.id,
                                                                    'DEVICE',
                                                                    'CONTAINS')
                            activo = relacion_from[0]._from

                            activo = rest_client.asset_controller.get_asset_by_id_using_get(
                                    asset_id=activo.id,
                                )
                            
                            logging.info(f"Además el dispositivo esta relacionado con el activo: {activo.name}")
                            pass
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene activo relacionado {e}")
                                          
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el dispositivo: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")

## 7. Creación de usuario
El correo de usuario es único en la base de datos.

In [ ]:
# Vamos a crear los usuarios asociados a cada customer (id_farmacia)
# Si ya existe el usuario vamos a listar a quien ya pertenece.
import csv
file_path = 'users_v2.csv'
DASHBOARD_COFA = "FARMACIAS"
ROLE_GROUP = "Customer User"
try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                email = datos.get("email", "")
                name = datos.get("nombre", "")
                last_name = datos.get("apellido", "")
                phone = datos.get("telefono", "")
                description = datos.get("descripcion", "")
                customer_relate = datos.get("relacion", "")
                
                try:
                    # 1.1 Intentar obtener el usuario por su correo, ya que debe ser único en AKIoT
                    
                    page_data = rest_client.user_controller.get_all_user_infos_using_get(page_size=4000, page=0, include_customers = True, text_search=email)

                    if page_data.data==[]:
                        logging.info(f"El usuario con correo electrónico  '{email}' no existe, se puede crear sin problemas.")
                        dashboard = rest_client.dashboard_controller.get_tenant_dashboards_using_get(page_size=4000, page=0, text_search=DASHBOARD_COFA)
                        dashboard = dashboard.data[0]
                        customer = rest_client.get_customers(page_size=1000, page=0,text_search=customer_relate)
                        customer = customer.data[0]
                        
                        # Fetching automatically created "Customer User" Group.
                        customer1_administrators = rest_client.get_entity_group_by_owner_and_name_and_type(customer.id, "USER", "Customer Users")
                        # Tengo que leer el dashboard id de farmacias COFA, uso una constante DASHBOARD_COFA y busco con este el id

                        # Creating User for Customer 1 with default dashboard from Tenant "Shared Dashboards" group.
                        user_email = email
                        user_password = "Akiot1234"
                        additional_info = {
                            "defaultDashboardId": dashboard.id.id,
                            "defaultDashboardFullscreen": False,
                            "description": description
                        }
                        user = User(authority="CUSTOMER_USER",
                                    customer_id=customer.id,
                                    email=user_email,
                                    first_name=name,
                                    last_name=last_name,
                                    phone=phone,
                                    additional_info=additional_info)
                        user = rest_client.save_user(user, send_activation_mail=False)
                        activation_link = rest_client.get_activation_link(user.id.id)
                        activate_token = activation_link.split('=')[-1][:-1]
                        rest_client.activate_user(body=ActivateUserRequest(activate_token, user_password),
                                                send_activation_mail=False)

                        rest_client.add_entities_to_entity_group(customer1_administrators.id, [user.id.id])
                        logging.info('User created:\n%r\n', user)

                        


                    else:
                        # Si ya existe el usuarui  vamos a listar a quien ya pertenece.
                        customer_id= page_data.data[0].customer_id
                        try: 
                            customer_name = rest_client.get_customer_by_id(customer_id)
                            logging.info(f"Este usuario ya existe y pertenece a customer con id_farmacia {customer_name.name}")
        
                        except Exception as e:
                            raise Exception(f"El usuario no esta asociado a ningun customer, pertenece al tenant actual {e}")
                        
                                          
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el Usuario: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")

Vamos a crear dispositivos nombre, perfil (leer el perfil creado o crear uno), Label (Queda a dispo del cliente)
Luego los atributos ser servidor cargo: address(direction de la farmacia), tipoSensor(Importante, define el state Dashboard), xPos, yPos.
, y se lo asigno al customer correspondiente, esto es importante ya que si no le asigno el dispositivo no lo podrá ver. Luego ese dispositivo también se tiene que lo tiene que relacionar con la Farmacia correspondiente, el numero de serie/nombre del HC5 se asocia con el numero de CUIT de la Farmacia

Armar el CSV que vincule esto. 



## 7. Reasignación de dispositivos por calibración

- Si intento crear un nuevo dispositivo y asignarle el mismo token de acceso de uno anterior, ThingsBoard devuelve un error.
- Primero debemos cambiar el token del dispositivo en cuestión. Este dispositivo puede identificarse a partir del token o del nombre + fecha_de_última_calibración.  
  Es necesario verificar qué dispositivo es, a qué farmacia pertenece (id_unico) y confirmar si ese último dueño es correcto. Conociendo el nuevo destino —a partir del informe recibido por la nueva farmacia— se procede a dar de alta el dispositivo con el formato **SN_nueva_fecha_de_calibración**.
- Una vez identificados los dispositivos, se solicita a ThingsBoard que asigne un token aleatorio (random) a los dispositivos anteriores. El token real, que está grabado en el firmware, se asigna al nuevo dispositivo (entidad) creado para la nueva farmacia.
- Con este esquema no se elimina la relación histórica del dispositivo con su farmacia anterior: se conserva todo el historial de dispositivos que pasaron por cada farmacia.


In [ ]:
# Vamos a crear los dispositivos asociados a cada activo (Farmacia) y customer (CUIT)
# Si ya existe el dispositivo vamos a listar a quien ya pertenece.
import csv
from tb_rest_client.models.models_ce import DeviceCredentials
file_path = 'dispositivos_v2_reasignados.csv'
name_device_profile = "HC5 NBIoT"
try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("name", "")
                tipo = datos.get("type", "")
                label = datos.get("label", "")
                token_acceso = datos.get("token_acceso", "")
                punto_monitoreo = datos.get("punto_monitoreo", "")
                relacion_farmacia = datos.get("relacion", "")

                # La dirección y el customer se obtienen de la farmacia(Activo)
                # address = datos.get("address", "")
                # customer = datos.get("customer", "")
                try:
                    
                    
                    # page_data = rest_client.get_user_devices(page_size=5000, page=0, text_search=name)
                    # Traigo todos los dipositivos
                    page_data = rest_client.get_user_devices(page_size=5000, page=0)
                    # para cada dispositivo interno busco cual ya tiene asignado el token de acceso que viene en el csv.
                    # imprimo el nombre del dispositivo y a que customer y a que farmacia pertenece este token.
                    for  device_i in page_data.data:
                        try:
                            device_credentials = rest_client.device_controller.get_device_credentials_by_device_id_using_get(device_i.id.id)
                            if device_credentials.credentials_id == token_acceso:
                                logging.info(f"El dispositivo con token de acceso '{token_acceso}' ya existe en el sistema.")
                                # Busco a que customer pertenece este dispositivo y que nombre tiene el dispositivo
                                customer_id= device_i.customer_id
                                try: 
                                    customer_name = rest_client.get_customer_by_id(customer_id)
                                    device_name = rest_client.get_device_by_id(device_credentials.device_id)
                                    logging.info(f"Este dispositivo ya existe, se llama {device_name.name} y pertenece a la farmacia con Identificador {customer_name.name}")
                
                                except Exception as e:
                                    raise Exception(f"El dispositivo no tiene customer, pertenece al tenant actual {e}")
                                
                                try:
                                    # Buscamos su relacion con algun activo
                                    relacion_from = rest_client.find_by_to_v1(device_i.id.id,
                                                                            'DEVICE',
                                                                            'CONTAINS')
                                    activo = relacion_from[0]._from

                                    activo = rest_client.asset_controller.get_asset_by_id_using_get(
                                            asset_id=activo.id,
                                        )
                                    
                                    logging.info(f"Además el dispositivo esta relacionado con el activo: {activo.name}")
                                    pass
                                except Exception as e:
                                    raise Exception(f"El dispositivo no tiene activo relacionado {e}")
                                # Para este dipositivo le creo una nueva credencia ramdon, de esta forma dejara de pertenecer a la farmacia anterior y procedere a crear el nuevo dispositivo en la nueva farmacia con el token correspondiente
                                try:

                                    new_access_token = "my_new_secret_token_123" # El nuevo token que quieres asignar

                                    updated_credentials_body = DeviceCredentials(
                                        id=device_credentials.id, # IMPORTANTE: Reutiliza el ID de las credenciales existentes
                                        device_id=device_credentials.device_id, # Reutiliza el ID del dispositivo
                                        credentials_type=device_credentials.credentials_type, # Reutiliza el tipo de credenciales (ej. ACCESS_TOKEN)
                                        credentials_id=new_access_token, # Aquí va el nuevo ACCESS_TOKEN
                                        credentials_value=device_credentials.credentials_value, # Para ACCESS_TOKEN, este campo suele ser None o una cadena vacía
                                        version=device_credentials.version # Puedes incluir la versión si es relevante para tu flujo
                                    )

                                    # 3. Llamar a la función para actualizar las credenciales
                                    credentials_new = rest_client.update_device_credentials(body=updated_credentials_body)
                                    # rest_client.device_controller.update_device_credentials_using_post(body = updated_credentials_body)
                                    logging.info(f"Credenciales del dispositivo actualizadas exitosamente:")
                                    print(updated_credentials_body)
                                    print(credentials_new)
                                except Exception as e:
                                    raise Exception(f"No se puedo cambiar el token porque: {e}")
                                    
                                
                        except Exception as e:
                            logging.error(f"Error al obtener las credenciales del dispositivo: {e}")
                    # get_device_credentials_by_device_id_using_get
                    # si no se encontró el dispositivo con esas credenciales imprimo y puedo seguir creando el nuevo entidad dispositivo.

                    # si lo encontré al dispositivo  a ese dispositivo le creo una nueva credencial con el token random, de esta forma dejara de pertenecer a la farmacia anterior y pasara a pertenecer a la nueva farmacia.

                    # update_device_credentials_using_post

                    # Ahora estamos en condiciones de crear el nuevo dispositivo como sigue normalmente.
                    
                    # 1.1 Intentar obtener el dispositivo por su nombre, ya que debe ser único en AKIoT
                    page_data = rest_client.get_user_devices(page_size=5000, page=0, text_search=name)
                    if page_data.data==[]:
                        logging.info(f"El dispositivo con nombre '{name}' no existe, se puede crear.")
                        # busco el perfil de dispositivo ya creado en AKIoT

                        device_profile = rest_client.device_profile_controller.get_device_profile_infos_using_get(page_size=100, page=0, text_search=name_device_profile)
                        if not device_profile.data:
                            raise Exception(f"No se encontró el perfil de dispositivo con nombre '{name_device_profile}'")
                        
                        
                        
                        asset_to_relate = rest_client.asset_controller.get_all_asset_infos_using_get(page_size=100, page=0,include_customers=True, text_search=relacion_farmacia)
                        # Tengo el customer al que asignare este dispositivo
                        customer_relate_id = asset_to_relate.data[0].customer_id
                        # Tengo la entidad a la que relacionare este dispositivo
                        new_device = Device(name=name, label=label, device_profile_id=device_profile.data[0].id,customer_id=customer_relate_id)
                        new_device = rest_client.save_device(new_device,access_token=token_acceso)
                        logging.info(" Device was created:\n%r\n", new_device)



                        asset_to_relate_id = asset_to_relate.data[0]
                        attributes_from_asset = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                                                                                            entity_type="ASSET",
                                                                                            entity_id=asset_to_relate_id.id,
                                                                                            scope="SERVER_SCOPE",
                                                                                            )
                        attributes_from_asset = {item['key']: item['value'] for item in attributes_from_asset}
                        # Ya tengo los atributos del dispositivo, address, punto_monitoreo, xPos,yPos
                        body_to_attributes = {
                            "address": attributes_from_asset['direccion'],
                            "punto_monitoreo": punto_monitoreo,
                            # "xPos": attributes_from_asset['latitud'],
                            # "yPos": attributes_from_asset['longitud'],
                        }
                        # Guardo los atributos del servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="DEVICE",
                                entity_id=new_device.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        # Creamos la relación entre el activo y el dispositivo
                        relation = EntityRelation(_from=asset_to_relate_id.id, to=new_device.id, type="Contains")
                        rest_client.save_relation(relation)
                        # Asignamos el dispositivo al customer (id_farmacia) correspondiente 

                        # En los activos Farmacia que el title sea el id_farmacia, asi en la lista de dispositivo me queda listado ee nombre de la id_farmacia a la que pertenece, es practico para el buscador
                    else:
                        # Si ya existe el dispositivo vamos a listar a quien ya pertenece.
                        customer_id= page_data.data[0].customer_id
                        try: 
                            customer_name = rest_client.get_customer_by_id(customer_id)
                            logging.info(f"Este dispositivo ya existe y pertenece a la farmacia con Identificador único {customer_name.name}")
        
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene customer, pertenece al tenant actual {e}")
                        
                        try:
                            # Buscamos su relacion con algun activo
                            relacion_from = rest_client.find_by_to_v1(page_data.data[0].id.id,
                                                                    'DEVICE',
                                                                    'CONTAINS')
                            activo = relacion_from[0]._from

                            activo = rest_client.asset_controller.get_asset_by_id_using_get(
                                    asset_id=activo.id,
                                )
                            
                            logging.info(f"Además el dispositivo esta relacionado con el activo: {activo.name}")
                            pass
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene activo relacionado {e}")
                                          
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el dispositivo: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")

2025-11-19 19:34:48 - INFO - 2539512668 - 39 - El dispositivo con token de acceso '282C0240F882' ya existe en el sistema.
2025-11-19 19:34:49 - INFO - 2539512668 - 45 - Este dispositivo ya existe, se llama AKIOT0F882_19_11_2025 y pertenece a la farmacia con Identificador 0005
2025-11-19 19:34:49 - INFO - 2539512668 - 61 - Además el dispositivo esta relacionado con el activo: Farmacia DEMO 5
2025-11-19 19:34:50 - ERROR - 2539512668 - 90 - Error al obtener las credenciales del dispositivo: No se puedo cambiar el token porque: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Length': '205', 'Content-Type': 'application/json;charset=ISO-8859-1', 'Date': 'Wed, 19 Nov 2025 19:34:49 GMT', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers'})
HTTP response body: b'{"status":400,"message":"Device credentials are already assigned to another device!","errorCode":31,"timestamp":1763580889956,"subscriptionErrorCode":null,"subscriptionEntry":null,